# 1-4,时间序列数据建模流程范例

2020年发生的新冠肺炎疫情灾难给各国人民的生活造成了诸多方面的影响。

有的同学是收入上的，有的同学是感情上的，有的同学是心理上的，还有的同学是体重上的。

本文基于中国2020年3月之前的疫情数据，建立时间序列RNN模型，对中国的新冠肺炎疫情结束时间进行预测。

![](./data/疫情前后对比.png)

In [ ]:
import torch

print("torch.__version__ = ", torch.__version__)



In [ ]:
import os

#mac系统上pytorch和matplotlib在jupyter中同时跑需要更改环境变量
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


## 一，准备数据

本文的数据集取自tushare，获取该数据集的方法参考了以下文章。

《https://zhuanlan.zhihu.com/p/109556102》

![](./data/1-4-新增人数.png)


In [ ]:
# 使用Jupyter Notebook魔法命令，将Matplotlib的图形内联显示在Notebook中
%matplotlib inline

# 配置图形输出格式为SVG，这将影响图形的显示质量
%config InlineBackend.figure_format = 'svg'

# 导入Pandas库并使用"pd"作为别名
import pandas as pd

# 导入Matplotlib库并使用"plt"作为别名
import matplotlib.pyplot as plt

# 使用Pandas的read_csv函数从文件中读取数据并存储在DataFrame对象中
# 文件路径为"./eat_pytorch_datasets/covid-19.csv"，分隔符为制表符("\t")
df = pd.read_csv("./eat_pytorch_datasets/covid-19.csv", sep="\t")

# 使用DataFrame的plot方法绘制折线图
# x轴数据为"date"列，y轴数据包括"confirmed_num"、"cured_num"和"dead_num"列
# 设置图形的尺寸为(8, 5)
df.plot(x="date", y=["confirmed_num", "cured_num", "dead_num"], figsize=(8, 5))

# 旋转x轴刻度标签，以免它们重叠在一起
plt.xticks(rotation=0)


In [ ]:
# 使用Pandas的set_index方法将DataFrame的索引设置为"date"列，以便按日期进行数据处理
dfdata = df.set_index("date")

# 使用Pandas的diff方法计算每日变化，并删除NaN值
dfdiff = dfdata.diff(periods=1).dropna()

# 将索引重置为"date"列，以便在后续绘图中使用日期作为x轴
dfdiff = dfdiff.reset_index("date")

# 使用DataFrame的plot方法创建每日变化的折线图
# x轴数据为"date"列，y轴数据包括"confirmed_num"、"cured_num"和"dead_num"列
# 设置图形的尺寸为(8, 5)
dfdiff.plot(x="date", y=["confirmed_num", "cured_num", "dead_num"], figsize=(8, 5))

# 旋转x轴刻度标签，以免它们重叠在一起
plt.xticks(rotation=0)

# 删除"date"列并将DataFrame的数据类型转换为float32
# 这通常用于提高数据的计算性能
dfdiff = dfdiff.drop("date", axis=1).astype("float32")


In [ ]:
dfdiff.head()

下面我们通过继承torch.utils.data.Dataset实现自定义时间序列数据集。

torch.utils.data.Dataset是一个抽象类，用户想要加载自定义的数据只需要继承这个类，并且覆写其中的两个方法即可：

* `__len__`:实现len(dataset)返回整个数据集的大小。
* `__getitem__`:用来获取一些索引的数据，使`dataset[i]`返回数据集中第i个样本。

不覆写这两个方法会直接返回错误。


In [ ]:
# 导入PyTorch中的数据集(Dataset)和数据加载器(DataLoader)模块
from torch.utils.data import Dataset, DataLoader

# 用某日前8天窗口数据作为输入来预测该日数据
WINDOW_SIZE = 8


# 自定义Covid19Dataset类，继承自PyTorch的Dataset类
class Covid19Dataset(Dataset):

    # 定义Dataset的长度，即数据样本的数量
    def __len__(self):
        # 数据集的长度为dfdiff的长度减去窗口大小（因为需要一定数量的历史数据来预测）
        return len(dfdiff) - WINDOW_SIZE

    # 定义如何获取数据样本
    def __getitem__(self, i):
        # 从DataFrame中提取历史数据作为输入特征
        x = dfdiff.loc[i:i + WINDOW_SIZE - 1, :]
        # 将特征数据转换为PyTorch张量
        feature = torch.tensor(x.values)
        # 获取该日的目标数据作为标签
        y = dfdiff.loc[i + WINDOW_SIZE, :]
        # 将标签数据转换为PyTorch张量
        label = torch.tensor(y.values)
        # 返回特征和标签的元组
        return (feature, label)


# 创建Covid19Dataset的实例，即数据集对象
ds_train = Covid19Dataset()

# 数据较小，可以将全部训练数据放入一个批次中，以提高性能
# 创建数据加载器对象，用于批量加载数据
# batch_size参数设置为38，表示将数据划分为大小为38的批次
dl_train = DataLoader(ds_train, batch_size=38)

# 遍历数据加载器，将第一个批次的数据提取出来
for features, labels in dl_train:
    break

# 将训练数据加载器同时用作验证数据加载器
dl_val = dl_train

## 二，定义模型

使用Pytorch通常有三种方式构建模型：使用nn.Sequential按层顺序构建模型，继承nn.Module基类构建自定义模型，继承nn.Module基类构建模型并辅助应用模型容器进行封装。

此处选择第二种方式构建模型。



In [ ]:
# 导入PyTorch库
import torch
# 导入PyTorch的神经网络模块
from torch import nn

# 设置随机种子以确保结果的可重复性
torch.random.seed()


# 定义一个自定义的神经网络块（Block）类
class Block(nn.Module):
    def __init__(self):
        super(Block, self).__init__()

    def forward(self, x, x_input):
        # 计算输出，采用ReLU激活函数
        x_out = torch.max((1 + x) * x_input[:, -1, :], torch.tensor(0.0))
        return x_out


# 定义一个神经网络类（Net）继承自PyTorch的nn.Module
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 定义一个LSTM层，包括输入大小为3，隐藏大小为3，5个堆叠的LSTM层，batch_first参数表示批次维度在第一维
        self.lstm = nn.LSTM(input_size=3, hidden_size=3, num_layers=5, batch_first=True)
        # 定义一个线性层，将LSTM的输出维度从3变换为3
        self.linear = nn.Linear(3, 3)
        # 创建一个自定义块的实例
        self.block = Block()

    def forward(self, x_input):
        # 将输入数据传递给LSTM层，取最后一个时间步的输出
        x = self.lstm(x_input)[0][:, -1, :]
        # 将LSTM的输出传递给线性层
        x = self.linear(x)
        # 将线性层的输出和输入数据传递给自定义块
        y = self.block(x, x_input)
        return y


# 创建神经网络的实例
net = Net()
# 打印网络的结构
print(net)


In [ ]:
# 导入torchkeras库中的summary函数
from torchkeras import summary

# 使用summary函数来查看神经网络模型的摘要信息
# 参数net是要查看的模型，input_data是输入数据，这里使用了之前加载的features
summary(net, input_data=features)


### 三，训练模型

训练Pytorch通常需要用户编写自定义训练循环，训练循环的代码风格因人而异。

有3类典型的训练循环代码风格：脚本形式训练循环，函数形式训练循环，类形式训练循环。

此处我们通过引入torchkeras库中的KerasModel工具来训练模型，无需编写自定义循环。

torchkeras详情:  https://github.com/lyhue1991/torchkeras 

注：循环神经网络调试较为困难，需要设置多个不同的学习率多次尝试，以取得较好的效果。



In [ ]:
# 导入torchmetrics库中的MeanAbsolutePercentageError指标
from torchmetrics.regression import MeanAbsolutePercentageError


# 定义均方百分比误差（Mean Squared Percentage Error，MSPE）函数
def mspe(y_pred, y_true):
    # 计算误差的百分比
    err_percent = (y_true - y_pred) ** 2 / (torch.max(y_true ** 2, torch.tensor(1e-7)))
    # 计算均值
    return torch.mean(err_percent)


# 创建神经网络模型实例
net = Net()

# 定义损失函数为MSPE
loss_fn = mspe

# 定义一个字典，其中包含要使用的度量指标
metric_dict = {"mape": MeanAbsolutePercentageError()}

# 定义优化器，这里使用Adam优化器，学习率为0.03，优化神经网络的参数
optimizer = torch.optim.Adam(net.parameters(), lr=0.03)

# 定义学习率调度器，这里使用StepLR调度器，每10个epoch将学习率乘以0.0001
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.0001)


In [ ]:
# 导入torchkeras库中的KerasModel类
from torchkeras import KerasModel

# 创建KerasModel实例，将神经网络模型(net)、损失函数(loss_fn)、度量指标(metrics_dict)、优化器(optimizer)和学习率调度器(lr_scheduler)传递给它
model = KerasModel(net,
                   loss_fn=loss_fn,
                   metrics_dict=metric_dict,
                   optimizer=optimizer,
                   lr_scheduler=lr_scheduler)


In [ ]:
# 使用model.fit方法来训练神经网络模型
dfhistory = model.fit(
    # 训练数据加载器
    train_data=dl_train,
    # 验证数据加载器
    val_data=dl_val,
    # 训练的总轮数（epochs）
    epochs=100,
    # 用于保存最佳模型的路径
    ckpt_path='checkpoint',
    # 当验证损失不再减小时的耐心期限，用于早停（early stopping）
    patience=10,
    # 用于监视的指标，这里使用验证损失
    monitor='val_loss',
    # 早停的模式，这里选择最小化验证损失
    mode='min',
    # 回调函数，用于自定义训练过程中的操作，这里设为None
    callbacks=None,
    # 是否绘制训练和验证曲线
    plot=True,
    # 是否在CPU上训练（如果为False，则使用GPU）
    cpu=True
)


### 四，评估模型

评估模型一般要设置验证集或者测试集，由于此例数据较少，我们仅仅可视化损失函数在训练集上的迭代情况。

In [ ]:
# 使用model.evaluate方法来评估神经网络模型在验证数据集(dl_val)上的性能
model.evaluate(dl_val)

### 五，使用模型

此处我们使用模型预测疫情结束时间，即 新增确诊病例为0 的时间。

In [ ]:
# 使用dfresult记录现有数据以及此后预测的疫情数据
# 从dfdiff中复制"confirmed_num"、"cured_num"和"dead_num"列的数据到dfresult
dfresult = dfdiff[["confirmed_num", "cured_num", "dead_num"]].copy()

# 打印dfresult的末尾几行数据
dfresult.tail()


In [ ]:
# 循环预测未来1000天的新增疫情走势
for i in range(1000):
    # 从dfresult中获取最近的38天数据，并将其转换为PyTorch张量
    arr_input = torch.unsqueeze(torch.from_numpy(dfresult.values[-38:, :]), axis=0)

    # 使用模型进行预测
    arr_predict = model.forward(arr_input)

    # 将预测结果转换为DataFrame，并设置列名为dfresult的列名
    dfpredict = pd.DataFrame(torch.floor(arr_predict).data.numpy(),
                             columns=dfresult.columns)

    # 将预测结果与dfresult连接，将其添加到dfresult中，并忽略索引
    dfresult = pd.concat([dfresult, dfpredict], ignore_index=True)


In [ ]:
# 查询dfresult DataFrame 中"confirmed_num"列的值为0的行，并显示前几行结果
dfresult.query("confirmed_num==0").head()

# 第50天开始新增确诊降为0，第45天对应3月10日，也就是5天后，即预计3月15日新增确诊降为0
# 注：该预测偏乐观


In [ ]:
dfresult.query("cured_num==0").head()
# 第137天开始新增治愈降为0，第45天对应3月10日，也就是大概3个月后，即6月12日左右全部治愈。
# 注: 该预测偏悲观，并且存在问题，如果将每天新增治愈人数加起来，将超过累计确诊人数。

### 六，保存模型

模型权重保存在了model.ckpt_path路径。

In [ ]:
print(model.ckpt_path)

In [ ]:
model.load_ckpt('checkpoint')  #可以加载权重